## **PROJECT BLUEPRINT**

**Project Goal:**

+ Build an end-to-end ETL pipeline that:

    + Extracts weather data from OpenWeather API

    + Transforms the raw data (cleaning, converting units, formatting)

    + Loads the cleaned data into PostgreSQL

+ Automates daily batch ingestion using cron / scheduled tasks

+ Visualizes current weather data + forecast weather data using BI Tool

           ┌──────────────────┐
           │ OpenWeather API  │
           └────────┬─────────┘
                    │ (JSON)
                    ▼
          ┌────────────────────┐
          │ Python ETL Script  │
          │ (requests, pandas) │
          └───────┬────────────┘
                  │ cleaned data
                  ▼
        ┌────────────────────────┐
        │   PostgreSQL Database  │
        └────────┬───────────────┘
                 │
                 ▼
       BI Tools / Dashboards


**Set up:** Python environment, PostgreSQL (create database, user, grant privillege..)

**Table 1:** Weather_current:

| Column      | Type      | Description        |
| ----------- | --------- | ------------------ |
| id          | SERIAL PK | Unique row         |
| city        | VARCHAR   | City name          |
| temp_c      | FLOAT     | Celsius            |
|feels_like   |FLOAT      | Celcius            |
|temps_min    |FLOAT      | Celcius            |
|temps_max    |FLOAT      | Celcius            |
|pressure     |FLOAT      |                    |
|grnd_level   |FLOAT      | Atmospheric pressure on the ground level |
| humidity    | INT       | Humidity %         |
| wind_speed  | FLOAT     | m/s                |
| description | VARCHAR   | Cloudy, Rainy, etc |
| dt          | TIMESTAMP | API timestamp      |
| created_at  | TIMESTAMP | Insertion time     |
|sunrise      |TIMESTAMP  | API timestamp      |
|sunset       |TIMESTAMP  |API timestamp       |


**Table 2:** Weather_forecast

| Column      | Type      | Description        |
| ----------- | --------- | ------------------ |
| id          | SERIAL PK | Unique row         |
| city        | VARCHAR   | City name          |
| temp_c      | FLOAT     | Celsius            |
|feels_like   |FLOAT      | Celcius            |
|temps_min    |FLOAT      | Celcius            |
|temps_max    |FLOAT      | Celcius            |
|pressure     |FLOAT      |                    |
|grnd_level   |FLOAT      | Atmospheric pressure on the ground level |
| humidity    | INT       | Humidity %         |
| wind_speed  | FLOAT     | m/s                |
| description | VARCHAR   | Cloudy, Rainy, etc |
| dt          | TIMESTAMP | API timestamp      |
| created_at  | TIMESTAMP | Insertion time     |
|sunrise      |TIMESTAMP  | API timestamp      |
|sunset       |TIMESTAMP  |API timestamp       |


**Table 3:** City information

| Column      | Type      | Description        |
| ----------- | --------- | ------------------ |
| id          | SERIAL PK | Unique row         |
| city        | VARCHAR   | City name          |
| country     | VARCHAR   | Country code       |
| lat         | FLOAT     |                    |
| lon         | FLOAT     |                    |
| timezone    | TIMESTAMP |                    |
|             |with zone  | API timestamp      |
| population  | INT       |                    |


## **Extract Phase**

In [1]:
import requests
import os

API_KEY = "57c2842bfac979c9a3e8b4297d85185d"

def fetch_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&lang=vi"

    try:
        response = requests.get(url)
        data = response.json()
        return data
    except Exception as e:
        print(f"Exception fetching {city}: {e}")
        return None
cities = [
    "Hanoi", "Sapa", "Ha Long", "Hai Duong",            # North
    "Hue", "Vinh", "Da Nang", "Quy Nhon", "Nha Trang", "Da Lat",   # Central
    "Ho Chi Minh", "Binh Duong", "Can Tho", "Vinh Long"            # South
]

# Loop through each city and print weather
for c in cities:
    result = fetch_weather(c)
    print(f"\n=== {c} ===")
    print(result)


=== Hanoi ===
{'coord': {'lon': 105.8412, 'lat': 21.0245}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'mây rải rác', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 295.15, 'feels_like': 294.37, 'temp_min': 295.15, 'temp_max': 295.15, 'pressure': 1021, 'humidity': 37, 'sea_level': 1021, 'grnd_level': 1020}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 355, 'gust': 5.89}, 'clouds': {'all': 34}, 'dt': 1764130277, 'sys': {'type': 1, 'id': 9308, 'country': 'VN', 'sunrise': 1764112440, 'sunset': 1764152047}, 'timezone': 25200, 'id': 1581130, 'name': 'Hà Nội', 'cod': 200}

=== Sapa ===
{'coord': {'lon': 120.3241, 'lat': 5.1544}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'mưa nhẹ', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 302.7, 'feels_like': 307.56, 'temp_min': 302.7, 'temp_max': 302.7, 'pressure': 1007, 'humidity': 72, 'sea_level': 1007, 'grnd_level': 1006}, 'visibility': 10000, 'wind': {'speed': 4.69, 'deg': 186, 'gust': 6.43}, 'rain': 

In [2]:
import requests
import os
import pandas as pd

def fetch_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=vi"

    try:
        response = requests.get(url)
        data = response.json()

        # Check if response is OK
        if data.get("cod") != 200:
            print(f"Error fetching {city}: {data.get('message')}")
            return None

        event = {
            "coord_lon": data['coord']['lon'],
            "coord_lat": data['coord']['lat'],

            "weather_id": data['weather'][0]['id'],
            "weather_main": data['weather'][0]['main'],
            "description": data['weather'][0]['description'],
            "weather_icon": data['weather'][0]['icon'],

            "base": data.get('base'),

            "temp": data['main']['temp'],
            "feels_like": data['main']['feels_like'],
            "temp_min": data['main']['temp_min'],
            "temp_max": data['main']['temp_max'],
            "pressure": data['main']['pressure'],
            "humidity": data['main']['humidity'],

            "visibility": data.get('visibility'),

            "wind_speed": data['wind']['speed'],
            "wind_deg": data['wind']['deg'],
            "wind_gust": data['wind'].get('gust'),

            "clouds_all": data['clouds']['all'],

            "dt": data['dt'],

            "country": data['sys']['country'],
            "sunrise": data['sys']['sunrise'],
            "sunset": data['sys']['sunset'],

            "timezone": data['timezone'],
            "city_id": data['id'],
            "city_name": data['name']
        }

        df_weather = pd.DataFrame([event])  # wrap in list to create DataFrame
        return df_weather

    except Exception as e:
        print(f"Exception fetching {city}: {e}")
        return None

cities = [
    "Hanoi", "Sapa", "Ha Long", "Hai Duong",            # North
    "Hue", "Vinh", "Da Nang", "Quy Nhon", "Nha Trang", "Da Lat",   # Central
    "Ho Chi Minh", "Binh Duong", "Can Tho", "Vinh Long"            # South
]

# Fetch weather for all cities and combine
df_current = pd.concat([fetch_weather(c) for c in cities if fetch_weather(c) is not None], ignore_index=True)

print(df_current.head())

   coord_lon  coord_lat  weather_id weather_main  description weather_icon  \
0   105.8412    21.0245         802       Clouds  mây rải rác          03d   
1   120.3241     5.1544         500         Rain      mưa nhẹ          10d   
2   107.0800    20.9511         803       Clouds      mây cụm          04d   
3   106.3167    20.9333         802       Clouds  mây rải rác          03d   
4   107.6000    16.4667         803       Clouds      mây cụm          04d   

       base   temp  feels_like  temp_min  ...  wind_deg  wind_gust  \
0  stations  22.00       21.22     22.00  ...       355       5.89   
1  stations  29.55       34.41     29.55  ...       186       6.43   
2  stations  23.25       22.62     23.25  ...       140       0.87   
3  stations  23.56       22.96     23.56  ...         1       4.43   
4  stations  21.06       21.52     21.06  ...       310        NaN   

   clouds_all          dt  country     sunrise      sunset  timezone  city_id  \
0          34  1764129959    

/tmp/ipykernel_28112/2654568134.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_current = pd.concat([fetch_weather(c) for c in cities if fetch_weather(c) is not None], ignore_index=True)


In [3]:
def fetch_forecast(city):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric&lang=vi"

    response = requests.get(url)
    data = response.json()
    return data

# Loop through each city and print weather
for c in cities:
    result = fetch_forecast(c)
    print(f"\n=== {c} ===")
    print(result)


=== Hanoi ===
{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1764136800, 'main': {'temp': 22.62, 'feels_like': 21.93, 'temp_min': 22.62, 'temp_max': 23.87, 'pressure': 1021, 'sea_level': 1021, 'grnd_level': 1018, 'humidity': 38, 'temp_kf': -1.25}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'mây rải rác', 'icon': '03d'}], 'clouds': {'all': 26}, 'wind': {'speed': 3.31, 'deg': 354, 'gust': 4.41}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-11-26 06:00:00'}, {'dt': 1764147600, 'main': {'temp': 23.45, 'feels_like': 22.74, 'temp_min': 23.45, 'temp_max': 24.17, 'pressure': 1019, 'sea_level': 1019, 'grnd_level': 1017, 'humidity': 34, 'temp_kf': -0.72}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'mây rải rác', 'icon': '03d'}], 'clouds': {'all': 38}, 'wind': {'speed': 2.42, 'deg': 19, 'gust': 4.11}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-11-26 09:00:00'}, {'dt': 1764158400, 'main': {'temp': 20, 'feels_like

In [4]:
import pandas as pd
def fetch_forecast(city):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric&lang=vi"
    response = requests.get(url)
    data = response.json()
    rows = []

    for item in data['list']:
        row = {
        'dt': item['dt'],
        'dt_txt': item['dt_txt'],
        'temp': item['main']['temp'],
        'feels_like': item['main']['feels_like'],
        'temp_min': item['main']['temp_min'],
        'temp_max': item['main']['temp_max'],
        'pressure': item['main']['pressure'],
        'sea_level': item['main'].get('sea_level'),
        'grnd_level': item['main'].get('grnd_level'),
        'humidity': item['main']['humidity'],
        'temp_kf': item['main'].get('temp_kf'),
        'weather_id': item['weather'][0]['id'],
        'weather_main': item['weather'][0]['main'],
        'description': item['weather'][0]['description'],
        'weather_icon': item['weather'][0]['icon'],
        'clouds_all': item['clouds']['all'],
        'wind_speed': item['wind']['speed'],
        'wind_deg': item['wind']['deg'],
        'wind_gust': item['wind'].get('gust'),
        'visibility': item.get('visibility'),
        'pop': item.get('pop'),
        'sys_pod': item['sys'].get('pod'),
        'city_id': data['city']['id'],
        'city_name': data['city']['name'],
        'country': data['city']['country'],
        'coord_lat': data['city']['coord']['lat'],
        'coord_lon': data['city']['coord']['lon'],
        'population': data['city'].get('population'),
        'timezone': data['city'].get('timezone'),
        'sunrise': data['city'].get('sunrise'),
        'sunset': data['city'].get('sunset')
    }
        rows.append(row)
    df_forecast = pd.DataFrame(rows)

    return df_forecast

# Loop through each city and print weather
df_forecasts = pd.concat([fetch_forecast(c) for c in cities], ignore_index=True)

print(df_forecasts.head())

           dt               dt_txt   temp  feels_like  temp_min  temp_max  \
0  1764136800  2025-11-26 06:00:00  22.62       21.93     22.62     23.87   
1  1764147600  2025-11-26 09:00:00  23.45       22.74     23.45     24.17   
2  1764158400  2025-11-26 12:00:00  20.00       19.18     20.00     20.00   
3  1764169200  2025-11-26 15:00:00  17.88       17.00     17.88     17.88   
4  1764180000  2025-11-26 18:00:00  16.99       16.15     16.99     16.99   

   pressure  sea_level  grnd_level  humidity  ...  sys_pod  city_id city_name  \
0      1021       1021        1018        38  ...        d  1581130    Hà Nội   
1      1019       1019        1017        34  ...        d  1581130    Hà Nội   
2      1022       1022        1021        43  ...        n  1581130    Hà Nội   
3      1024       1024        1024        49  ...        n  1581130    Hà Nội   
4      1024       1024        1023        54  ...        n  1581130    Hà Nội   

  country coord_lat  coord_lon  population  timezo

## **Transform data**

In [5]:
#process time column
from datetime import datetime
# Convert UNIX timestamps for current weather
df_current["dt"] = pd.to_datetime(df_current["dt"], unit="s")
df_current["sunrise"] = pd.to_datetime(df_current["sunrise"], unit="s")
df_current["sunset"] = pd.to_datetime(df_current["sunset"], unit="s")

# Convert UNIX timestamps for forecast weather
df_forecasts["dt"] = pd.to_datetime(df_forecasts["dt"], unit="s")
df_forecasts["sunrise"] = pd.to_datetime(df_forecasts["sunrise"], unit="s")
df_forecasts["sunset"] = pd.to_datetime(df_forecasts["sunset"], unit="s")

# Display columns
print(df_current[["dt", "sunrise", "sunset"]].head())
print(df_forecasts[["dt", "sunrise", "sunset"]].head())

                   dt             sunrise              sunset
0 2025-11-26 04:05:59 2025-11-25 23:14:00 2025-11-26 10:14:07
1 2025-11-26 04:13:12 2025-11-25 21:50:27 2025-11-26 09:41:47
2 2025-11-26 04:09:33 2025-11-25 23:08:55 2025-11-26 10:09:18
3 2025-11-26 04:10:11 2025-11-25 23:11:56 2025-11-26 10:12:23
4 2025-11-26 04:08:41 2025-11-25 22:59:14 2025-11-26 10:14:49
                   dt             sunrise              sunset
0 2025-11-26 06:00:00 2025-11-25 23:14:00 2025-11-26 10:14:07
1 2025-11-26 09:00:00 2025-11-25 23:14:00 2025-11-26 10:14:07
2 2025-11-26 12:00:00 2025-11-25 23:14:00 2025-11-26 10:14:07
3 2025-11-26 15:00:00 2025-11-25 23:14:00 2025-11-26 10:14:07
4 2025-11-26 18:00:00 2025-11-25 23:14:00 2025-11-26 10:14:07


In [6]:
df_current["dt"] = df_current["dt"].dt.tz_localize("UTC").dt.tz_convert("Asia/Ho_Chi_Minh")
df_current["sunrise"] = df_current["sunrise"].dt.tz_localize("UTC").dt.tz_convert("Asia/Ho_Chi_Minh")
df_current["sunset"] = df_current["sunset"].dt.tz_localize("UTC").dt.tz_convert("Asia/Ho_Chi_Minh")
print(df_current[["dt", "sunrise", "sunset"]].head())

                         dt                   sunrise  \
0 2025-11-26 11:05:59+07:00 2025-11-26 06:14:00+07:00   
1 2025-11-26 11:13:12+07:00 2025-11-26 04:50:27+07:00   
2 2025-11-26 11:09:33+07:00 2025-11-26 06:08:55+07:00   
3 2025-11-26 11:10:11+07:00 2025-11-26 06:11:56+07:00   
4 2025-11-26 11:08:41+07:00 2025-11-26 05:59:14+07:00   

                     sunset  
0 2025-11-26 17:14:07+07:00  
1 2025-11-26 16:41:47+07:00  
2 2025-11-26 17:09:18+07:00  
3 2025-11-26 17:12:23+07:00  
4 2025-11-26 17:14:49+07:00  


In [7]:
df_forecasts["dt"] = df_forecasts["dt"].dt.tz_localize("UTC").dt.tz_convert("Asia/Ho_Chi_Minh")
df_forecasts["sunrise"] = df_forecasts["sunrise"].dt.tz_localize("UTC").dt.tz_convert("Asia/Ho_Chi_Minh")
df_forecasts["sunset"] = df_forecasts["sunset"].dt.tz_localize("UTC").dt.tz_convert("Asia/Ho_Chi_Minh")
print(df_forecasts[["dt", "sunrise", "sunset"]].head())

                         dt                   sunrise  \
0 2025-11-26 13:00:00+07:00 2025-11-26 06:14:00+07:00   
1 2025-11-26 16:00:00+07:00 2025-11-26 06:14:00+07:00   
2 2025-11-26 19:00:00+07:00 2025-11-26 06:14:00+07:00   
3 2025-11-26 22:00:00+07:00 2025-11-26 06:14:00+07:00   
4 2025-11-27 01:00:00+07:00 2025-11-26 06:14:00+07:00   

                     sunset  
0 2025-11-26 17:14:07+07:00  
1 2025-11-26 17:14:07+07:00  
2 2025-11-26 17:14:07+07:00  
3 2025-11-26 17:14:07+07:00  
4 2025-11-26 17:14:07+07:00  


## **Load data**

In [8]:
import sqlalchemy
print(sqlalchemy.__version__)

1.4.54


In [9]:
from sqlalchemy import create_engine
import psycopg2

conn = psycopg2.connect(
    dbname="weather_data",
    user="postgres",     # change if needed
    password="050701",  # change
    host="localhost",
    port=5432
)

cursor = conn.cursor()

In [10]:
df_cities = df_current[["city_id", "city_name", "country", "coord_lat", "coord_lon", "timezone"]].drop_duplicates()

insert_query = """
    INSERT INTO cities (city_id, city_name, country, coord_lat, coord_lon, timezone)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (city_id) DO NOTHING  -- optional if city_id is PK to avoid duplicates
"""

for row in df_cities.itertuples(index=False):
    cursor.execute(insert_query, row)

conn.commit()

In [11]:
except_column = ["city_name", "country", "coord_lat", "coord_lon", "timezone", "population"]

df_current = df_current[[col for col in df_current.columns if col not in except_column]]
df_current.head()

,weather_id,weather_main,description,weather_icon,base,temp,feels_like,temp_min,temp_max,pressure,humidity,visibility,wind_speed,wind_deg,wind_gust,clouds_all,dt,sunrise,sunset,city_id
0,802,Clouds,mây rải rác,03d,stations,22.00,21.22,22.00,22.00,1021,37,10000,4.12,355,5.89,34,2025-11-26 11:05:59+07:00,2025-11-26 06:14:00+07:00,2025-11-26 17:14:07+07:00,1581130
1,500,Rain,mưa nhẹ,10d,stations,29.55,34.41,29.55,29.55,1007,72,10000,4.69,186,6.43,99,2025-11-26 11:13:12+07:00,2025-11-26 04:50:27+07:00,2025-11-26 16:41:47+07:00,1687343
2,803,Clouds,mây cụm,04d,stations,23.25,22.62,23.25,23.25,1020,38,10000,0.99,140,0.87,70,2025-11-26 11:09:33+07:00,2025-11-26 06:08:55+07:00,2025-11-26 17:09:18+07:00,1580410
3,802,Clouds,mây rải rác,03d,stations,23.56,22.96,23.56,23.56,1021,38,10000,4.13,1,4.43,49,2025-11-26 11:10:11+07:00,2025-11-26 06:11:56+07:00,2025-11-26 17:12:23+07:00,1581326
4,803,Clouds,mây cụm,04d,stations,21.06,21.52,21.06,21.06,1018,88,10000,5.14,310,NaN,75,2025-11-26 11:08:41+07:00,2025-11-26 05:59:14+07:00,2025-11-26 17:14:49+07:00,1580240


In [12]:
df_current.columns

Index(['weather_id', 'weather_main', 'description', 'weather_icon', 'base',
       'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'visibility', 'wind_speed', 'wind_deg', 'wind_gust', 'clouds_all', 'dt',
       'sunrise', 'sunset', 'city_id'],
      dtype='object')

In [13]:
df_forecasts = df_forecasts[[col for col in df_forecasts.columns if col not in except_column]]
df_forecasts.head()

,dt,dt_txt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,...,clouds_all,wind_speed,wind_deg,wind_gust,visibility,pop,sys_pod,city_id,sunrise,sunset
0,2025-11-26 13:00:00+07:00,2025-11-26 06:00:00,22.62,21.93,22.62,23.87,1021,1021,1018,38,...,26,3.31,354,4.41,10000,0.0,d,1581130,2025-11-26 06:14:00+07:00,2025-11-26 17:14:07+07:00
1,2025-11-26 16:00:00+07:00,2025-11-26 09:00:00,23.45,22.74,23.45,24.17,1019,1019,1017,34,...,38,2.42,19,4.11,10000,0.0,d,1581130,2025-11-26 06:14:00+07:00,2025-11-26 17:14:07+07:00
2,2025-11-26 19:00:00+07:00,2025-11-26 12:00:00,20.00,19.18,20.00,20.00,1022,1022,1021,43,...,31,2.75,38,6.41,10000,0.0,n,1581130,2025-11-26 06:14:00+07:00,2025-11-26 17:14:07+07:00
3,2025-11-26 22:00:00+07:00,2025-11-26 15:00:00,17.88,17.00,17.88,17.88,1024,1024,1024,49,...,6,3.05,19,5.03,10000,0.0,n,1581130,2025-11-26 06:14:00+07:00,2025-11-26 17:14:07+07:00
4,2025-11-27 01:00:00+07:00,2025-11-26 18:00:00,16.99,16.15,16.99,16.99,1024,1024,1023,54,...,8,3.44,339,7.39,10000,0.0,n,1581130,2025-11-26 06:14:00+07:00,2025-11-26 17:14:07+07:00


In [14]:
df_forecasts.columns

Index(['dt', 'dt_txt', 'temp', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'sea_level', 'grnd_level', 'humidity', 'temp_kf',
       'weather_id', 'weather_main', 'description', 'weather_icon',
       'clouds_all', 'wind_speed', 'wind_deg', 'wind_gust', 'visibility',
       'pop', 'sys_pod', 'city_id', 'sunrise', 'sunset'],
      dtype='object')

In [15]:
# Load current weather
insert_query = """
INSERT INTO current_weather (
    weather_id, weather_main, description, base,
    temp, feels_like, temp_min, temp_max, pressure, humidity,
    visibility, wind_speed, wind_deg, wind_gust, clouds_all, dt,
    sunrise, sunset, city_id
) VALUES (
    %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s,
    %s, %s, %s
)

"""

columns_order = [
    'weather_id', 'weather_main', 'description', 'base',
    'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
    'visibility', 'wind_speed', 'wind_deg', 'wind_gust', 'clouds_all', 'dt',
    'sunrise', 'sunset', 'city_id'
]

df_current_ordered = df_current[columns_order]

try:
    for row in df_current_ordered.itertuples(index=False):
        cursor.execute(insert_query, row)
    conn.commit()  # commit all successful inserts
except Exception as e:
    print(f"Transaction failed: {e}")
    conn.rollback()  # rollback everything

In [16]:
# Load current weather
insert_query = """
INSERT INTO forecast_weather (
    dt, dt_txt, temp, feels_like, temp_min, temp_max,
       pressure, humidity,
       weather_id, weather_main, description,
       wind_speed, wind_deg, wind_gust, visibility,
        city_id, sunrise, sunset
) VALUES (
    %s, %s, %s, 
    %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s,
    %s, %s, %s
)

"""

columns_order = [
    'dt', 'dt_txt', 'temp', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity',
       'weather_id', 'weather_main', 'description',
       'wind_speed', 'wind_deg', 'wind_gust', 'visibility',
        'city_id', 'sunrise', 'sunset'
]

df_current_ordered = df_forecasts[columns_order]

try:
    for row in df_current_ordered.itertuples(index=False):
        cursor.execute(insert_query, row)
    conn.commit()  # commit all successful inserts
except Exception as e:
    print(f"Transaction failed: {e}")
    conn.rollback()  # rollback everything

In [17]:
cursor.close()
conn.close()

**END.**